# Báo cáo Đồ án Song song hóa và tối ưu hóa một ứng dụng
- Thông tin nhóm
  - Lê Thanh Hiếu - 1712434
  - Nguyễn Văn Tuấn - 1512638
  - Phan Hữu Tú - 1712861

- Đề tài: **Thay đổi kích thước ảnh bằng Seam Carving**

In [ ]:
# import image file from drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/My Drive/LTSSUD/gpu-python')
!ls

images	__pycache__  README.md	report_cpu.ipynb  seam_carving.py  test.py


In [ ]:
from IPython.display import Image

In [ ]:
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)

## 1. Mô tả ứng dụng
- **Input**: một tấm ảnh (RGB)
- **Output**: một tấm ảnh mới được thay đổi kích thước (không chỉ cho chiều rộng mà còn cho chiều cao, không chỉ thu nhỏ mà còn phóng to) từ tấm ảnh gốc mà không làm biến dạng các đối tượng quan trọng.
   
- **Ý nghĩa thực tế khi sử dụng?**
  
   Ý nghĩa vô cùng to lớn trong các hệ thống lớn, nhất là hệ thống web mà hiển thị nhiều sản phẩm ảnh, ví dụ chúng ta sẽ check vào đường mạng của người dùng mà render ảnh đúng với tốc độ đường truyền đó, ví dụ với tốc độ của 3G thì ta sẽ response ảnh với size nhỏ hơn, tuỳ vào đường truyền mà ta sẽ cho người dùng sự trải nghiệm loading trên web tốt nhất. vây câu hỏi đặt ra chúng ta sẽ làm gì tất nhiên là resize ảnh(auto) seam carving giúp chúng ta làm điều đó tốt hơn bao giờ hết. 1 ví dụ khác khi tạo thubnail cho 1 video chúng ta sẽ dựa vào nhưng yếu tố khác để resize ảnh về đúng khung hình ta mong muốn như màn hình điện thoại 16:9 , 3:4
- Ứng dụng này có cần tăng tốc không?
  Vì seam carving được cài đặt theo quy hoạch động tốn rất nhiều time tính toán.
- Cải tiến và mở rộng
  +  Triển khai nhanh hơn trên GPU.
  + Đa toán tử: Kết hợp với cắt xén và chia tỷ lệ.
  + Loại bỏ nhiều đường nối nhanh hơn nhiều

## 2. Cài đặt tuần tự


### 2.1. Các bước triển khai thuật toán
- **Ý tưởng**: việc thay đổi kích thước ảnh được thực hiện bằng cách loại bỏ đi các điểm ảnh ít quan trọng và giữ lại các điểm ảnh quan trọng.




#### Workflow
![](https://github.com/kalpeshdusane/Seam-Carving-B.E.-Project/raw/master/readmeImage/flowchart.png)

> Nguồn: https://github.com/kalpeshdusane/Seam-Carving-B.E.-Project


#### Thuật toán xóa seam theo cột
  - Bước 1: Chuyển ảnh từ RGB sang Grayscale
  - Bước 2: Tính bảng Energy của ảnh
  - Bước 3: Tìm bảng chi phí nhỏ nhất
  - Bước 4: Tìm seam có chi phí nhỏ nhất
  - Bước 5: Xóa seam nhỏ nhất tìm được ở bước 4
  - Bước 6: Lặp lại bước 5 cho tới khi xóa  đủ số seams cần xóa


#### Thuật toán xóa seam theo hàng
  - Bước 1: Xoay ảnh một góc 90 độ theo chiều kim đồng hồ
  - Bước 2: Lặp lại thuật toán xóa seam theo cột cho đến khi xóa đủ số seams cần xóa
  - Bước 3: Xoay ảnh lại một góc 90 độ ngược chiều kim đồng hồ

#### Thuật toán chèn seam theo cột
  - Bước 1: Thực hiện thuật toán xóa seam theo cột trên một ảnh được sao chép từ ảnh gốc và lưu lại danh sách các điểm ảnh đã xóa theo đúng thứ tự đã xóa
  - Bước 2: Chèn các seams vào ảnh gốc theo đúng thứ tự tọa độ đã lưu. Với giá trị của pixel được chèn được lấy bằng trung bình của 2 pixels kế bên.

#### Thuật toán chèn seam theo hàng
  - Bước 1: Xoay ản 90 độ theo chiều kim đồng hồ
  - Bước 2: Thực hiện thuật toán chèn seam theo cột cho đến khi đủ số seams cần xóa
  - Bước 3: Xoay ảnh lại một góc 90 độ ngược chiều kim đồng hồ

### 2.2. Chi tiết thuật toán

#### Xây dựng bảng Energy


#### Xây dựng bảng chi phí nhỏ nhất
- Sử dụng Dynamic Programming để xây dựng bảng chi phí nhỏ nhất
    - Ở hàng trên cùng ta có tổng chi phí giữ nguyên<br/><br/><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/74/DynamicProgrammingLeastEnergyPathA.svg/840px-DynamicProgrammingLeastEnergyPathA.svg.png" width="400">  
    - Với các hàng còn lại, chi phí của mỗi pixel bằng engergy của nó cộng với chi phí nhỏ nhất của 3 pixel liền kề phía trên. Lặp lại cho đến hàng cuối cùng<br/><br/><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/c2/DynamicProgrammingLeastEnergyPathB.svg/840px-DynamicProgrammingLeastEnergyPathB.svg.png" width="400">

#### Xác định seam nhỏ nhất
- Từ bảng chi phí nhỏ nhất trên ta xác định được vị trí có tổng chi phí nhỏ nhất ở hàng cuối cùng và quay lui để xác định seam nhỏ nhất<br/><br/><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/05/DynamicProgrammingLeastEnergyPathC.svg/840px-DynamicProgrammingLeastEnergyPathC.svg.png" width="400">

## 3. Cài đặt song song

## 4. Tối ưu hóa cài đặt song song

## 5. Nhìn lại quá trình làm đồ án

## 6. Tài liệu tham khảo
- https://vnoi.info/wiki/cs/imageprocessing/Seam-Carving.md
- https://en.wikipedia.org/wiki/Seam_carving#cite_note-ACM1276390-1


- Song song
  - https://shwestrick.github.io/2020/07/29/seam-carve.html